In [1]:
import networkx as nx

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def build_fat_tree(k):
    G = nx.Graph()
    pods = k
    core_switches = [f"Core_{i}" for i in range((k//2)**2)]
    for p in range(pods):
        aggs = [f"Pod{p}_Agg_{i}" for i in range(k//2)]
        edges = [f"Pod{p}_Edge_{i}" for i in range(k//2)]
        for e in edges:
            for a in aggs:
                G.add_edge(e, a) 
        for e_idx, e in enumerate(edges):
             for s in range(k//2):
                server_name = f"Pod{p}_Server_{e_idx}_{s}"
                G.add_edge(e, server_name)
        # Standard Fat-Tree wiring: Agg switch 'i' in Pod 'p' connects to 
        # Core switches starting at index 'i * (k/2)'
        stride = k // 2
        for i, a in enumerate(aggs):
            for c_idx in range(i * stride, (i + 1) * stride):
                G.add_edge(a, core_switches[c_idx]) 
    return G

def run_analysis(k, link_speed_gbps=10):
    G = build_fat_tree(k)
    print(f"FAT-TREE ANALYSIS (k={k})")
    # Intra Pod Analysis
    intra_src = f"Pod0_Server_0_0" 
    intra_dst = f"Pod0_Server_1_0"
    intra_paths = list(nx.all_shortest_paths(G, source=intra_src, target=intra_dst))
    print(f"[Intra-Pod] Paths between {intra_src} and {intra_dst}:")
    print(f"   -> Found {len(intra_paths)} minimal paths.")
    for p in intra_paths: print("   ", p)
    print()
    # Inter Pod Analysis
    inter_src = f"Pod0_Server_0_0" 
    inter_dst = f"Pod1_Server_0_0"
    inter_paths = list(nx.all_shortest_paths(G, source=inter_src, target=inter_dst))
    print(f"[Inter-Pod] Paths between {inter_src} and {inter_dst}:")
    print(f"   -> Found {len(inter_paths)} minimal paths.")
    for p in inter_paths: print("   ", p)
    print()
    # Bisection Bandwidth
    total_servers = (k**3) / 4
    bisection_links = (k**3) / 8
    bisection_bw = bisection_links * link_speed_gbps
    print(f"[Bisection Bandwidth]")
    print(f"   -> Total Servers: {int(total_servers)}")
    print(f"   -> Bisection Width (Links): {int(bisection_links)}")
    print(f"   -> Link Speed: {link_speed_gbps} Gbps")
    print(f"   -> CALCULATED BISECTION BANDWIDTH: {int(bisection_bw)} Gbps")
run_analysis(k=4, link_speed_gbps=10)

FAT-TREE ANALYSIS (k=4)
[Intra-Pod] Paths between Pod0_Server_0_0 and Pod0_Server_1_0:
   -> Found 2 minimal paths.
    ['Pod0_Server_0_0', 'Pod0_Edge_0', 'Pod0_Agg_0', 'Pod0_Edge_1', 'Pod0_Server_1_0']
    ['Pod0_Server_0_0', 'Pod0_Edge_0', 'Pod0_Agg_1', 'Pod0_Edge_1', 'Pod0_Server_1_0']

[Inter-Pod] Paths between Pod0_Server_0_0 and Pod1_Server_0_0:
   -> Found 4 minimal paths.
    ['Pod0_Server_0_0', 'Pod0_Edge_0', 'Pod0_Agg_0', 'Core_0', 'Pod1_Agg_0', 'Pod1_Edge_0', 'Pod1_Server_0_0']
    ['Pod0_Server_0_0', 'Pod0_Edge_0', 'Pod0_Agg_0', 'Core_1', 'Pod1_Agg_0', 'Pod1_Edge_0', 'Pod1_Server_0_0']
    ['Pod0_Server_0_0', 'Pod0_Edge_0', 'Pod0_Agg_1', 'Core_2', 'Pod1_Agg_1', 'Pod1_Edge_0', 'Pod1_Server_0_0']
    ['Pod0_Server_0_0', 'Pod0_Edge_0', 'Pod0_Agg_1', 'Core_3', 'Pod1_Agg_1', 'Pod1_Edge_0', 'Pod1_Server_0_0']

[Bisection Bandwidth]
   -> Total Servers: 16
   -> Bisection Width (Links): 8
   -> Link Speed: 10 Gbps
   -> CALCULATED BISECTION BANDWIDTH: 80 Gbps
